# Create Binary Models with Ellipsoidal Inclusions

This notebook demonstrates the `create_binary_model` function for generating synthetic binary models with ellipsoidal inclusions. These models are useful for:

- Testing image processing algorithms
- Validation of elastic property calculations
- Educational demonstrations
- Synthetic dataset generation

## Key Features

- **Homogeneous background** with ellipsoidal inclusions
- **Flexible positioning**: Random or explicit placement
- **Shape control**: Spheres, oblate (flattened), or prolate (elongated) ellipsoids
- **Orientation control**: Fixed or random rotation
- **Digital rock convention**: Background=1 (solid), Inclusion=0 (pore)

## Import Required Libraries

In [ ]:
from drp_template.tools import create_binary_model
from drp_template.input_output import export_model
from drp_template.image import ortho_views, save_figure
from drp_template.math import get_phase_fractions
from drp_template.default_params import print_style
import numpy as np
import matplotlib.pyplot as plt

## Example 1: Single Centered Sphere

Create a simple model with one spherical pore inclusion centered in the volume. Using explicit positioning ensures the sphere is visible in all orthogonal views.

In [ ]:
print_style("Example 1: Single Spherical Inclusion (Centered)", style='section')

# Create model with centered sphere using explicit position
nx, ny, nz = 100, 100, 100
center_position = np.array([[nx // 2, ny // 2, nz // 2]])  # Centered at (50, 50, 50)

data = create_binary_model(
    nx=nx, ny=ny, nz=nz,
    num_inclusions=1,
    inclusion_radius=20,
    inclusion_aspect_ratio=1.0,  # Sphere
    orientation='zx',
    random_orientation=False,
    # Default: background_value=1 (Solid), inclusion_value=0 (Pore)
    dtype='uint8',
    positions=center_position  # Explicit center position
)

print(f"Created model with shape: {data.shape}")
print(f"Unique values: {np.unique(data)}")
print(f"Center position: {center_position[0]}")

In [ ]:
# Export with metadata
labels = {0: 'Pore', 1: 'Solid'}
export_model(
    filename='binary_single_sphere',
    data=data,
    voxel_size=10.0,
    dtype='uint8',
    order='C',
    filetype='.raw',
    labels=labels
)

# Analyze phases
get_phase_fractions(data, labels=labels, log=True)

In [ ]:
# Visualize orthogonal views
fig, axs = ortho_views(
    data,
    paramsfile='binary_single_sphere.json',
    labels=labels,
    title='Single Spherical Inclusion'
)
plt.show()

## Example 2: Multiple Ellipsoidal Inclusions

Create a model with multiple prolate (elongated) ellipsoids with random orientations. This example inverts the phase values to show solid grains in a pore matrix.

In [ ]:
print_style("Example 2: Multiple Ellipsoidal Inclusions", style='section')

# Create model - using inverted values to show solid inclusions in pore matrix
data = create_binary_model(
    nx=150, ny=150, nz=150,
    num_inclusions=15,
    inclusion_radius=12,
    inclusion_aspect_ratio=1.5,  # Prolate ellipsoid (elongated)
    orientation='xy',
    random_orientation=True,  # Random rotations
    background_value=0,  # Pore space
    inclusion_value=1,   # Solid grains (explicitly inverted from default)
    dtype='uint8',
    seed=123
)

print(f"Created model with shape: {data.shape}")
print(f"Unique values: {np.unique(data)}")

# Export and analyze
labels = {0: 'Pore', 1: 'Solid Grain'}
export_model(
    filename='binary_multi_ellipsoids',
    data=data,
    voxel_size=5.0,
    dtype='uint8',
    order='C',
    filetype='.raw',
    labels=labels
)

get_phase_fractions(data, labels=labels, log=True)

In [ ]:
# Visualize
fig, axs = ortho_views(
    data,
    paramsfile='binary_multi_ellipsoids.json',
    labels=labels,
    title='Multiple Ellipsoidal Inclusions',
)
plt.show()

## Example 3: Oblate (Flattened) Inclusions

Create a model with oblate ellipsoids (flattened, disk-like pores). The aspect ratio < 1 creates flattened shapes.

**Key concept**: 
- `orientation='xy'` → Ellipse in XY plane, flattened along Z-axis
- `orientation='zx'` → Ellipse in ZX plane, flattened along Y-axis  
- `orientation='zy'` → Ellipse in ZY plane, flattened along X-axis

The oblate ellipsoids are **NOT** single-voxel thick - they have a thickness determined by the radius and aspect ratio.

In [ ]:
print_style("Example 3: Oblate (Flattened) Inclusions", style='section')

# Create model with default values (solid=1, pores=0)
# Using orientation='xy' so ellipses are in XY plane, flattened along Z
data = create_binary_model(
    nx=120, ny=120, nz=120,
    num_inclusions=12,
    inclusion_radius=15,
    inclusion_aspect_ratio=0.5,  # Oblate (flattened, aspect < 1)
    orientation='xy',  # Ellipse in XY plane, thin in Z direction
    random_orientation=False,
    dtype='uint8',
    seed=456
)

print(f"Created model with shape: {data.shape}")
print(f"With orientation='xy' and aspect_ratio=0.5:")
print(f"  - Wide dimension (radius): {15} voxels in X and Y")
print(f"  - Thin dimension: {15 * 0.5:.1f} voxels in Z")
print(f"  → Disk-like pores, flattened along Z-axis")

# Export and analyze
labels = {0: 'Pore', 1: 'Rock Matrix'}
export_model(
    filename='binary_oblate_grains',
    data=data,
    voxel_size=8.0,
    dtype='uint8',
    order='C',
    filetype='.raw',
    labels=labels
)

get_phase_fractions(data, labels=labels, log=True)

# Visualize
fig, axs = ortho_views(
    data,
    paramsfile='binary_oblate_grains.json',
    labels=labels,
    title='Oblate Inclusions (XY plane, thin in Z)',
    slice_indices=(50, 50, 50)
)
plt.show()

print("\n💡 Note: In the visualization:")
print("   - XY view: Shows wide circular cross-sections")
print("   - YZ view: Shows thin edge views (elongated along Y)")
print("   - XZ view: Shows thin edge views (elongated along X)")

### Understanding Orientation and Thickness

Let's create the same oblate ellipsoid with different orientations to see the effect:

In [ ]:
# Create a centered oblate ellipsoid to clearly see the shape
nx, ny, nz = 80, 80, 80
center = np.array([[nx // 2, ny // 2, nz // 2]])
radius = 25
aspect = 0.3  # Very flat for clear visualization

# Test different orientations
orientations = ['xy', 'zx', 'zy']

for orient in orientations:
    data_test = create_binary_model(
        nx=nx, ny=ny, nz=nz,
        num_inclusions=1,
        inclusion_radius=radius,
        inclusion_aspect_ratio=aspect,
        orientation=orient,
        random_orientation=False,
        dtype='uint8',
        positions=center
    )
    
    # Calculate actual dimensions
    wide_dim = radius
    thin_dim = radius * aspect
    
    print(f"\n📐 Orientation: '{orient}'")
    if orient == 'xy':
        print(f"   Ellipse in XY plane → Wide in X({wide_dim}) & Y({wide_dim}), Thin in Z({thin_dim:.1f})")
    elif orient == 'zx':
        print(f"   Ellipse in ZX plane → Wide in Z({wide_dim}) & X({wide_dim}), Thin in Y({thin_dim:.1f})")
    elif orient == 'zy':
        print(f"   Ellipse in ZY plane → Wide in Z({wide_dim}) & Y({wide_dim}), Thin in X({thin_dim:.1f})")
    
    # Show counts in each direction
    print(f"   Pore voxels per slice: XY={np.sum(data_test[:,:,nz//2]==0)}, "
          f"YZ={np.sum(data_test[nx//2,:,:]==0)}, "
          f"XZ={np.sum(data_test[:,ny//2,:]==0)}")

### Visual Comparison

The key insight: **Oblate ellipsoids are 3D objects with finite thickness**, not 2D planes!

With `aspect_ratio=0.5` and `radius=15`:
- **Wide dimensions**: 15 voxels (full radius)
- **Thin dimension**: 15 × 0.5 = **7.5 voxels** (still many voxels!)

So they're "pancakes" or "disks", not infinitely thin sheets.

## Example 4: Explicit Positioning

Demonstrate full control over inclusion placement by specifying exact coordinates. This creates 4 spheres positioned at the corners of a cube.

In [ ]:
print_style("Example 4: Explicit Positioning", style='section')

# Define explicit positions for 4 inclusions in a 3D grid pattern
nx, ny, nz = 120, 120, 120
positions = np.array([
    [30, 30, 30],   # Bottom-left-front
    [90, 30, 30],   # Bottom-right-front
    [30, 90, 90],   # Top-left-back
    [90, 90, 90],   # Top-right-back
])

data = create_binary_model(
    nx=nx, ny=ny, nz=nz,
    num_inclusions=4,
    inclusion_radius=15,
    inclusion_aspect_ratio=1.0,  # Spheres
    orientation='zx',
    random_orientation=False,
    dtype='uint8',
    positions=positions  # Use explicit positions
)

print(f"Created model with shape: {data.shape}")
print(f"Inclusion positions:\n{positions}")

In [ ]:
# Export and analyze
labels = {0: 'Pore', 1: 'Rock Matrix'}
export_model(
    filename='binary_explicit_positions',
    data=data,
    voxel_size=10.0,
    dtype='uint8',
    order='C',
    filetype='.raw',
    labels=labels
)

get_phase_fractions(data, labels=labels, log=True)

# Visualize
fig, axs = ortho_views(
    data,
    paramsfile='binary_explicit_positions.json',
    labels=labels,
    title='Explicit Positioning (4 Corners)',
    slice_indices=(30, 35, 90)
)
plt.show()

## Advanced: Custom Position Patterns

You can create various spatial patterns using NumPy:

In [ ]:
# 1. Linear array along x-axis
positions_linear = np.linspace([20, 50, 50], [100, 50, 50], num=5).astype(int)
print("Linear array positions:")
print(positions_linear)

# 2. Regular 3D grid
x, y, z = np.meshgrid([30, 60, 90], [30, 60, 90], [30, 60, 90])
positions_grid = np.column_stack([x.ravel(), y.ravel(), z.ravel()])
print(f"\n3D grid positions ({len(positions_grid)} inclusions):")
print(positions_grid[:6])  # Show first 6

# 3. Random positions (saved for reproducibility)
np.random.seed(42)
positions_random = np.random.randint(20, 100, size=(8, 3))
print("\nRandom positions:")
print(positions_random)

## Parameter Summary

Key parameters and their effects:

| Parameter | Values | Effect |
|-----------|--------|--------|
| `inclusion_aspect_ratio` | `1.0` | Sphere (all dimensions equal) |
| | `< 1.0` | Oblate (flattened, disk-like) |
| | `> 1.0` | Prolate (elongated, cigar-like) |
| `orientation` | `'xy'` | Ellipse in XY plane, flattened along Z |
| | `'zx'` | Ellipse in ZX plane, flattened along Y |
| | `'zy'` | Ellipse in ZY plane, flattened along X |
| `random_orientation` | `True` / `False` | Random 3D rotation per inclusion |
| `periodic` | `True` / `False` | Enable periodic boundary conditions |
| `background_value` | `1` (default) | Rock matrix / solid |
| `inclusion_value` | `0` (default) | Pore space |
| `positions` | `None` | Random placement |
| | `array (N, 3)` | Explicit coordinates |
| `seed` | `int` | Reproducible random placement |

**Important Notes**:
- With `aspect_ratio=0.5` and `radius=15`, the thin dimension is still **7.5 voxels thick** - not a single voxel!
- When `periodic=True`, inclusions near boundaries wrap to opposite side (seamless tiling)
- Periodic boundaries create up to 27 copies (1 original + 6 faces + 12 edges + 8 corners)

## Viewing in ParaView

To visualize your exported `.raw` files in ParaView:

1. **Open** the `.raw` file in ParaView
2. **Choose** "Image Reader"
3. **Set Data Extent**: `0 (nx-1) 0 (ny-1) 0 (nz-1)`
   - Example: For 100×100×100 → `0 99 0 99 0 99`
4. **Set Data Scalar Type**: **unsigned char** (for `dtype='uint8'`)
5. **Set Data Byte Order**: Little Endian
6. **Apply** and visualize!

See the [ParaView Guide](../../docs/tutorials/paraview_guide.md) for detailed instructions.

## Summary

This notebook demonstrated:

✅ **Creating binary models** with spherical and ellipsoidal pore inclusions  
✅ **Shape control** via aspect ratio (sphere, oblate, prolate)  
✅ **Random placement** with reproducible seeding  
✅ **Explicit positioning** for precise spatial control  
✅ **Export with metadata** including labels and voxel size  
✅ **Phase fraction analysis** to quantify porosity  
✅ **3D visualization** with orthogonal views  

### Next Steps

- Combine with `import_model()` to process real rock data
- Use synthetic models to validate image processing algorithms
- Calculate effective elastic properties on these models
- Generate training datasets for machine learning

### See Also

- [Tools API Reference](../../docs/api/tools.md)
- [Input/Output API Reference](../../docs/api/input_output.md)
- [ParaView Visualization Guide](../../docs/tutorials/paraview_guide.md)

## Example 5: Periodic Boundary Conditions (RVE)

Periodic boundary conditions create seamless, tileable volumes where inclusions near boundaries wrap to the opposite side. This is essential for:

- **Representative Volume Elements (RVE)** in computational homogenization
- **Periodic unit cells** for material property calculations
- **Seamless tiling** without edge artifacts
- **Finite element simulations** requiring periodic BCs

When `periodic=True`, inclusions that extend beyond domain boundaries are automatically replicated at corresponding periodic positions.

In [ ]:
print_style("Example 5a: Single Inclusion Near Boundary (No PBC)", style='section')

# Create a model with a large inclusion near the corner WITHOUT periodic boundaries
nx, ny, nz = 100, 100, 100
corner_position = np.array([[15, 15, 15]])  # Near corner

data_no_pbc = create_binary_model(
    nx=nx, ny=ny, nz=nz,
    num_inclusions=1,
    inclusion_radius=25,  # Large enough to extend beyond boundaries
    inclusion_aspect_ratio=1.0,
    orientation='zx',
    random_orientation=False,
    periodic=False,  # No periodic boundaries
    dtype='uint8',
    positions=corner_position
)

print(f"Model shape: {data.shape}")
print(f"Inclusion position: {corner_position[0]}")
print(f"Inclusion radius: 25 voxels")
print(f"Without PBC: Inclusion is cut off at boundaries")

# Analyze boundary faces
face_x0 = data_no_pbc[0, :, :]
face_x1 = data_no_pbc[-1, :, :]
pores_x0 = np.sum(face_x0 == 0)
pores_x1 = np.sum(face_x1 == 0)

print(f"\nBoundary analysis:")
print(f"  Pores on X=0 face: {pores_x0}")
print(f"  Pores on X=max face: {pores_x1}")
print(f"  → Inclusion only visible on X=0 face (cut off, no wrapping)")

In [ ]:
print_style("Example 5b: Same Inclusion WITH Periodic Boundaries", style='section')

# Same configuration but WITH periodic boundaries
data_pbc = create_binary_model(
    nx=nx, ny=ny, nz=nz,
    num_inclusions=1,
    inclusion_radius=25,
    inclusion_aspect_ratio=1.0,
    orientation='zx',
    random_orientation=False,
    periodic=True,  # Enable periodic boundaries
    dtype='uint8',
    positions=corner_position
)

print(f"Model shape: {data_pbc.shape}")
print(f"With PBC: Inclusion wraps to opposite boundaries")

# Analyze all 6 boundary faces
face_x0_pbc = data_pbc[0, :, :]
face_x1_pbc = data_pbc[-1, :, :]
face_y0_pbc = data_pbc[:, 0, :]
face_y1_pbc = data_pbc[:, -1, :]
face_z0_pbc = data_pbc[:, :, 0]
face_z1_pbc = data_pbc[:, :, -1]

pores_x0_pbc = np.sum(face_x0_pbc == 0)
pores_x1_pbc = np.sum(face_x1_pbc == 0)
pores_y0_pbc = np.sum(face_y0_pbc == 0)
pores_y1_pbc = np.sum(face_y1_pbc == 0)
pores_z0_pbc = np.sum(face_z0_pbc == 0)
pores_z1_pbc = np.sum(face_z1_pbc == 0)

print(f"\nBoundary analysis (all 6 faces):")
print(f"  X=0 face:   {pores_x0_pbc} pore voxels")
print(f"  X=max face: {pores_x1_pbc} pore voxels ✓ (wrapped)")
print(f"  Y=0 face:   {pores_y0_pbc} pore voxels")
print(f"  Y=max face: {pores_y1_pbc} pore voxels ✓ (wrapped)")
print(f"  Z=0 face:   {pores_z0_pbc} pore voxels")
print(f"  Z=max face: {pores_z1_pbc} pore voxels ✓ (wrapped)")
print(f"\n✅ Inclusion appears on all 6 boundary faces (seamless wrapping)")

In [ ]:
# Visualize comparison: PBC vs No PBC
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Show boundary slices where wrapping is visible
slice_idx = 0  # Boundary slice

# No PBC - boundary slices
axes[0, 0].imshow(data_no_pbc[slice_idx, :, :].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[0, 0].set_title(f'No PBC: YZ slice (x={slice_idx})')
axes[0, 0].set_xlabel('y')
axes[0, 0].set_ylabel('z')

axes[0, 1].imshow(data_no_pbc[:, slice_idx, :].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[0, 1].set_title(f'No PBC: XZ slice (y={slice_idx})')
axes[0, 1].set_xlabel('x')
axes[0, 1].set_ylabel('z')

axes[0, 2].imshow(data_no_pbc[:, :, slice_idx].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[0, 2].set_title(f'No PBC: XY slice (z={slice_idx})')
axes[0, 2].set_xlabel('x')
axes[0, 2].set_ylabel('y')

# With PBC - same boundary slices showing wrapping
axes[1, 0].imshow(data_pbc[slice_idx, :, :].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[1, 0].set_title(f'With PBC: YZ slice (x={slice_idx})\n[Wrapped from x=max]')
axes[1, 0].set_xlabel('y')
axes[1, 0].set_ylabel('z')

axes[1, 1].imshow(data_pbc[:, slice_idx, :].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[1, 1].set_title(f'With PBC: XZ slice (y={slice_idx})\n[Wrapped from y=max]')
axes[1, 1].set_xlabel('x')
axes[1, 1].set_ylabel('z')

axes[1, 2].imshow(data_pbc[:, :, slice_idx].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[1, 2].set_title(f'With PBC: XY slice (z={slice_idx})\n[Wrapped from z=max]')
axes[1, 2].set_xlabel('x')
axes[1, 2].set_ylabel('y')

plt.tight_layout()
plt.show()

print("\n💡 Compare top row (No PBC) vs bottom row (With PBC):")
print("   Bottom row shows inclusion appearing on opposite boundaries!")

In [ ]:
print_style("Example 5c: Multiple Inclusions with Periodic Boundaries", style='section')

# Create RVE with multiple random inclusions and periodic boundaries
data_rve = create_binary_model(
    nx=120, ny=120, nz=120,
    num_inclusions=10,
    inclusion_radius=10,
    inclusion_aspect_ratio=1.2,  # Slightly elongated
    orientation='xy',             # Ignored when random_orientation=True
    random_orientation=True,      # Enable TRUE 3D Euler rotations
    periodic=True,                # Periodic boundaries for RVE
    dtype='uint8',
    seed=21
)

print(f"RVE Model shape: {data_rve.shape}")
print(f"Number of inclusions: 10")
print(f"Random orientations: True")
print(f"Periodic boundaries: True")

# Export as RVE
labels = {0: 'Pore', 1: 'Rock Matrix'}
export_model(
    filename='binary_rve_periodic',
    data=data_rve,
    voxel_size=5.0,
    dtype='uint8',
    order='C',
    filetype='.raw',
    labels=labels
)

get_phase_fractions(data_rve, labels=labels, log=True)

# Visualize
fig, axs = ortho_views(
    data_rve,
    paramsfile='binary_rve_periodic.json',
    labels=labels,
    title='RVE with Periodic Boundaries',
    slice_indices=(59, 60, 61)
)
plt.show()

print("\n✅ This RVE can be seamlessly tiled in all directions!")
print("   Perfect for finite element simulations with periodic BCs.")

### Demonstrating Seamless Tiling

Let's verify the periodic boundaries by creating a 2×2×2 tiled array:

In [ ]:
# Create a 2×2×2 tiled version of the periodic RVE
tiled = np.block([
    [
        [data_pbc, data_pbc],
        [data_pbc, data_pbc]
    ],
    [
        [data_pbc, data_pbc],
        [data_pbc, data_pbc]
    ]
])

print(f"Original RVE size: {data_pbc.shape}")
print(f"Tiled 2×2×2 size: {tiled.shape}")

# Show XY slices to demonstrate seamless tiling
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Show slice at z=0 (boundary)
slice_z = 0
axes[0].imshow(tiled[:, :, slice_z].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[0].set_title(f'Tiled 2×2: XY slice (z={slice_z})')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
# Mark the boundaries
axes[0].axhline(ny, color='red', linewidth=2, linestyle='--', alpha=0.7, label='Tile boundaries')
axes[0].axvline(nx, color='red', linewidth=2, linestyle='--', alpha=0.7)
axes[0].legend()

# Show slice in the middle
slice_z_mid = nz
axes[1].imshow(tiled[:, :, slice_z_mid].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[1].set_title(f'Tiled 2×2: XY slice (z={slice_z_mid})')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
# Mark the boundaries
axes[1].axhline(ny, color='red', linewidth=2, linestyle='--', alpha=0.7, label='Tile boundaries')
axes[1].axvline(nx, color='red', linewidth=2, linestyle='--', alpha=0.7)
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n✅ Notice: No visible seams at the red boundaries!")
print("   The periodic wrapping creates perfectly seamless tiling.")